In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from test import evaluate_test_set
import sts_data
from importlib import reload
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score
import matplotlib as plt

2022-02-25 18:08:48.653045: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-25 18:08:48.653072: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import pandas as pd
from preprocess import Preprocess
import logging
import torch
import re
from dataset import STSDataset
from datasets import load_dataset,Dataset
import spacy
import torchtext
from torchtext.data import get_tokenizer
from spacy.lang.en import English
from torchtext.legacy.data import Field
from torch.utils.data import DataLoader
import numpy as np

In [4]:
# dataset = load_dataset('text', data_files='SICK.txt')
# dataset
columns_mapping = {
        "sent1": "sentence_A",
        "sent2": "sentence_B",
        "label": "relatedness_score",
    }
stopwords_path="stopwords-en.txt"
columns_mapping.values()

dict_values(['sentence_A', 'sentence_B', 'relatedness_score'])

In [5]:
sick_dataset = load_dataset('sick',download_mode='reuse_cache_if_exists')
sick_dataset=sick_dataset.remove_columns(['label','id','entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'])
train_pd=pd.DataFrame.from_dict(sick_dataset['train'])
validation_pd=pd.DataFrame.from_dict(sick_dataset['validation'])
test_pd=pd.DataFrame.from_dict(sick_dataset['test'])
sick_data=[train_pd,validation_pd,test_pd]
# sick_df = pd.DataFrame(data=sick_dataset.data, columns=sick_dataset.column_names)
# sen_A=columns_mapping['sent1']
# sen_B= columns_mapping['sent2']
# score=columns_mapping['label']
# sick_df=sick_df[[sen_A,sen_B,score]]
# pd.read_csv(dataset)

Dataset sick downloaded and prepared to /home/ibrahimssd/.cache/huggingface/datasets/sick/default/0.0.0/c6b3b0b44eb84b134851396d6d464e5cb8f026960519d640e087fe33472626db. Subsequent calls will reuse this data.


In [6]:
# sick_dataset['train']
# Dataset.from_pandas(tran_pd)
splits= list(sick_dataset.keys())
type(sick_dataset)
len(train_data_df)
4439/64

NameError: name 'train_data_df' is not defined

In [ ]:
import re
import string

"""
Performs basic text cleansing on the unstructured field 
"""


class Preprocess:
    def __init__(self, stpwds_file_path):
        """
        Initializes regex patterns and loads stopwords
        """
        # TODO implement
        with open(stpwds_file_path) as fh:
            self.stopwords=list(set(fh.read().split()))
        self.noise_re = re.compile('\\b(%s)\\W'%('|'.join(map(re.escape,self.stopwords))),re.I)
    

    def perform_preprocessing(self, data, columns_mapping):
        sen_A=columns_mapping['sent1']
        sen_B= columns_mapping['sent2']
        score=columns_mapping['label']
        cleaned_data=[]
        for data_frame in data:
            groupA=list(data_frame[sen_A])
            groupB=list(data_frame[sen_B])
            ## normalize text to lower case
            groupA=[x.lower() for x in groupA]
            groupB=[x.lower() for x in groupB]
            ## remove punctuations
            groupA=[''.join(c for c in x if c not in string.punctuation) for x in groupA]
            groupB=[''.join(c for c in x if c not in string.punctuation) for x in groupB]
            ## remove stopwords
            groupA=[self.noise_re.sub('',p) for p in groupA]
            groupB=[self.noise_re.sub('',p) for p in groupB]
            # Trim extra whitespace
            groupA=[' '.join(x.split()) for x in groupA]
            groupB=[' '.join(x.split()) for x in groupB]
            # Remove numbers
            groupA=[''.join(c for c in x if c not in '0123456789') for x in groupA]
            groupB=[''.join(c for c in x if c not in '0123456789') for x in groupB]
            ## return data_back to DataFrame
            data_frame[sen_A]=groupA
            data_frame[sen_B]=groupB
            cleaned_data.append(data_frame)
        
        
        sick_dataset={'train':Dataset.from_pandas(cleaned_data[0]),
                      'validation':Dataset.from_pandas(cleaned_data[1]),
                      'test':Dataset.from_pandas(cleaned_data[2])}
        data_frame=pd.concat(cleaned_data, ignore_index=True)
        
        return sick_dataset


In [ ]:
process=Preprocess(stopwords_path)
formatted_data=process.perform_preprocessing(sick_data,columns_mapping)


In [ ]:
train_data= pd.DataFrame(formatted_data['train'])

In [ ]:
cols = list(columns_mapping.values())
cols.pop()
sen_list=list(train_data[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1))
sen_str = ' '.join(map(str, sen_list))
print(len(sen_list))

In [ ]:
# Word tokenization
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()
text = """When learning data science, you shouldn't get discouraged!
Challenges and setbacks aren't failures, they're just part of the journey. You've got this!"""

#  "nlp" Object is used to create documents with linguistic annotations.
my_doc = nlp(sen_str)

# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)
# print(token_list)

In [ ]:
from torchtext.vocab import vocab
from collections import Counter, OrderedDict
counter = Counter(token_list)
sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
v= vocab(ordered_dict)

PAD_token = 0   # Used for padding short sentences
SOS_token = 1   # Start-of-sentence token
EOS_token = 2   # End-of-sentence token
special_words={PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
default_index = -1
v.set_default_index(default_index)
for key , value in special_words.items():
    if value not in v: v.insert_token(value, key)



In [ ]:
with open(stopwords_path) as fh:
            stopwords=list(set(fh.read().split()))

In [ ]:
train_data['sentenceA&B']=train_data[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [ ]:
text_field = Field(
#     tokenize='basic_english', 
    lower=True,
    include_lengths=True,
    pad_token='PAD',
    pad_first='SOS',
    stop_words=stopwords,
)

# label_field = Field(sequential=False, use_vocab=False)
preprocessed_text = train_data['sentenceA&B'].apply(lambda x: text_field.preprocess(x))
text_field.build_vocab(
    preprocessed_text, 
    vectors='fasttext.simple.300d')

# get the vocab instance
vocab = text_field.vocab

In [ ]:
preprocessed_text
len(vocab.itos)

In [ ]:
# def data2tensors(self, data):
#         """
#         Converts raw data sequences into vectorized sequences as tensors
#         """
#         pass
#         # TODO implement

# def get_data_loader(self, batch_size=8):
#         pass
#         # TODO implement

# def sort_batch(self, batch, targets, lengths):
#         """
#         Sorts the data, lengths and target tensors based on the lengths
#         of the sequences from longest to shortest in batch
#         """
#         sents1_lengths, perm_idx = lengths.sort(0, descending=True)
#         sequence_tensor = batch[perm_idx]
#         target_tensor = targets[perm_idx]
#         return sequence_tensor.transpose(0, 1), target_tensor, sents1_lengths

# def vectorize_sequence(self, sentence):
#         """
#         Replaces tokens with their indices in vocabulary
#         """
#         pass
#         # TODO implement

# def pad_sequences(self, vectorized_sents, sents_lengths):
#         """
#         Pads zeros at the end of each sequence in data tensor till max
#         length of sequence in that batch
#         """
#         pass
#         # TODO implement
# def encode(vocab,string):
#        return vocab[string]
        

In [ ]:
# train_data['sentenceA&B'][0].apply(lambda x: print(x) )

In [ ]:
def vectorize_sequence(sentence):
        """
        Replaces tokens with their indices in vocabulary
        """
        splited_sentence=sentence.split()
        encodes=[vocab[token] for token in splited_sentence]
        return encodes

In [ ]:
cols = list(columns_mapping.values())
cols.pop()
train_data= pd.DataFrame(formatted_data['train'])
val_data=pd.DataFrame(formatted_data['validation'])
test_data=pd.DataFrame(formatted_data['test'])
train_data['sentenceA&B']=train_data[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
val_data['sentenceA&B']=val_data[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
test_data['sentenceA&B']=test_data[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

codes=[]
for sentence in val_data['sentenceA&B']:
    encodes= vectorize_sequence(sentence)
    codes.append(encodes)
val_data['sentenceA&B'] = codes
val_data

In [ ]:
def data2tensors(data):
        """
        Converts raw data sequences into vectorized sequences as tensors
        """
        
        
        
        data['sent1_tensor']=data['sent1_tensor'].apply(lambda lis : torch.as_tensor(lis))
        data['sent2_tensor']=data['sent2_tensor'].apply(lambda lis : torch.as_tensor(lis))
           
        
        
        return data
    

In [ ]:
# pd.DataFrame(data2tensors(formatted_data)['train'])["sentenceA&B"]

In [ ]:
dataloaders = {x:DataLoader(formatted_data[x],32, shuffle=True, num_workers=4) for x in ['train','validation','test']}

In [ ]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

# train_features, train_labels = next(iter(dataloaders['train']))
# from torch.utils import data
# train_tensor = data.TensorDataset(train_data, train_label)

In [ ]:
# from torch.utils import data
# train_tensor = data.TensorDataset(train_data, train_label)

In [ ]:
# dataloaders = {'train': torch.utils.data.DataLoader(STSDataset(sts_train_df, batch_size=64)),
#                'val': torch.utils.data.DataLoader(STSDataset(sts_dev_df, batch_size=64))}


In [ ]:
# train_target = torch.tensor(train['Target'].values.astype(np.float32))
# train = torch.tensor(train.drop('Target', axis = 1).values.astype(np.float32)) 
# train_tensor = data_utils.TensorDataset(train, train_target) 
# train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [ ]:
train_data_df= pd.DataFrame(formatted_data['train'])
val_data_df=pd.DataFrame(formatted_data['validation'])
test_data_df=pd.DataFrame(formatted_data['test'])

In [ ]:
train_data_df['sent1_tensor']=train_data_df['sentence_A'].apply(lambda sen: vectorize_sequence(sen))
train_data_df['sent2_tensor']=train_data_df['sentence_B'].apply(lambda sen: vectorize_sequence(sen))

In [ ]:
# train_data_df = torch.tensor(train_data_df['sentence_A'].values.astype(np.float32))
train_data_df=data2tensors(train_data_df)
train_data_df['sents1_length_tensor']=train_data_df['sent1_tensor'].apply(lambda tensor : torch.tensor(len(tensor)))
train_data_df['sents2_length_tensor']=train_data_df['sent2_tensor'].apply(lambda tensor : torch.tensor(len(tensor)))

In [ ]:
#import pad_sequences
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# padded=pad_sequences(train_data_df['sent1_tensor'],padding="post",truncating=”post”,maxlen=8)


In [ ]:
def pad_sequences(sequences):
        """
        :param sequences: list of tensors
        :return:
        """
        num = len(sequences)
        max_len = max([s.shape[0] for s in sequences])
        out_dims = (num, max_len, *sequences[0].shape[1:])
        out_tensor = sequences[0].data.new(*out_dims).fill_(0)
        mask = sequences[0].data.new(*out_dims).fill_(0)
        for i, tensor in enumerate(sequences):
            length = tensor.size(0)
            out_tensor[i, :length] = tensor
            mask[i, :length] = 1
        return list(out_tensor)

In [ ]:
train_data_df['sent1_tensor']=pad_sequences(train_data_df['sent1_tensor'])
train_data_df['sent2_tensor']=pad_sequences(train_data_df['sent2_tensor'])

In [ ]:
train_data_df['relatedness_score']=train_data_df['relatedness_score'].apply(lambda score : torch.tensor(score/sum(train_data_df['relatedness_score'])))

In [ ]:
"""
Standard Pytorch Dataset class for loading datasets.
"""


class STSDataset(Dataset):
    def __init__(
        self,
        sent1_tensor,
        sent2_tensor,
        target_tensor,
        sents1_length_tensor,
        sents2_length_tensor,
        raw_sents_1,
        raw_sents_2,
    ):
        """
        initializes  and populates the the length, data and target tensors, and raw texts list
        """
        
        assert (
            
            
            len(sent1_tensor)
            == torch.tensor(list(target_tensor)).size(0)
            == len(sent2_tensor)
            == torch.tensor(list(sents1_length_tensor)).size(0)
            == torch.tensor(list(sents2_length_tensor)).size(0)
        )
        self.sent1_tensor = sent1_tensor
        self.sent2_tensor = sent2_tensor
        self.target_tensor = target_tensor
        self.sents1_length_tensor = sents1_length_tensor
        self.sents2_length_tensor = sents2_length_tensor
        self.raw_sents_1 = raw_sents_1
        self.raw_sents_2 = raw_sents_2

    def __getitem__(self, index):
        """
        returns the tuple of data tensor, targets, lengths of sequences tensor and raw texts list
        """
        return (
            self.sent1_tensor[index],
            self.sent2_tensor[index],
            self.sents1_length_tensor[index],
            self.sents2_length_tensor[index],
            self.target_tensor[index],
            self.raw_sents_1[index],
            self.raw_sents_2[index],
        )

    def __len__(self):
        """
        returns the length of the data tensor.
        """
        return torch.tensor(list(self.target_tensor)).size(0)


In [ ]:
train_data_df                       

In [ ]:
train_dataset=  STSDataset(train_data_df['sent1_tensor'],
                         train_data_df['sent2_tensor'],
                         train_data_df['relatedness_score'],
                         train_data_df['sents1_length_tensor'],
                         train_data_df['sents2_length_tensor'],
                         train_data_df['sentence_A'],
                         train_data_df['sentence_B']
                         )

train_loader = DataLoader(dataset = train_dataset, batch_size = 64, shuffle = True)

In [ ]:
# len(train_data_df['sent1_tensor'])
# train_data_df['sents1_length_tensor'].size()
# torch.tensor(list(target_tensor)).size(0)
# train_data_df['sent1_tensor'][0]
# train_data_df['relatedness_score'][0]
# train_data_df['sents1_length_tensor'][0]
# train_data_df['sentence_A'][0]

In [ ]:
train_data_df['relatedness_score'][0].size()

In [ ]:
train_loader

In [ ]:
dataiter = iter(train_loader)
len(dataiter.next())

In [ ]:
for batch in train_loader:
    print(batch)
    break

In [ ]:
batch[0]

In [ ]:
n, d, m = 3, 5, 7
embedding = torch.nn.Embedding(n, d, max_norm=True)
W = torch.randn((m, d), requires_grad=True)
idx = torch.tensor([1, 2])
# a = embedding.weight.clone() @ W.t()  # weight must be cloned for this to be differentiable
# b = embedding(idx) @ W.t()  # modifies weight in-place
# out = (a.unsqueeze(0) + b.unsqueeze(1))
# loss = out.sigmoid().prod()
# loss.backward()

In [ ]:
embedding(idx)

In [ ]:
vocab_size= len(vocab)
embedding_size= 300

In [ ]:
embedding = torch.nn.Embedding(vocab_size, embedding_size, max_norm=True)

In [ ]:
sen=torch.tensor([[121,  82,   5,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 36,   4, 404,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [ ]:
embedding(sen).size()

In [ ]:
vocab.vectors[sen]

In [ ]:
embedding_weights= vocab.vectors
net = torch.nn.LSTM(10, 20, 1)

In [ ]:
def similarity_score(input1, input2):
    # Get similarity predictions:
    dif = input1.squeeze() - input2.squeeze()

    norm = torch.norm(dif, p=1, dim=dif.dim() - 1)
    y_hat = torch.exp(-norm)
    y_hat = torch.clamp(y_hat, min=1e-7, max=1.0 - 1e-7)
    return y_hat


In [ ]:
from sts_data import STSData

columns_mapping = {
        "sent1": "sentence_A",
        "sent2": "sentence_B",
        "label": "relatedness_score",
    }
dataset_name = "sick"
sick_data = STSData(
    dataset_name=dataset_name,
    columns_mapping=columns_mapping,
    normalize_labels=True,
    normalization_const=5.0,
)
batch_size = 64
sick_dataloaders = sick_data.get_data_loader(batch_size=batch_size)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from utils import similarity_score


"""
Wrapper class using Pytorch nn.Module to create the architecture for our model
Architecture is based on the paper: 
A STRUCTURED SELF-ATTENTIVE SENTENCE EMBEDDING
https://arxiv.org/pdf/1703.03130.pdf
"""


class SiameseBiLSTMAttention(nn.Module):
    def __init__(
        self,
        batch_size,
        output_size,
        hidden_size,
        vocab_size,
        embedding_size,
        embedding_weights,
        lstm_layers,
        device,
        bidirectional,
        self_attention_config,
        fc_hidden_size,
    ):
        super(SiameseBiLSTMAttention, self).__init__()
        """
        Initializes model layers and loads pre-trained embeddings from task 1
        """
        ## model hyper parameters
        self.batch_size = batch_size
        self.output_size = output_size
        self.lstm_hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.lstm_layers = lstm_layers
        self.device = device
        self.bidirectional = bidirectional
        self.fc_hidden_size = fc_hidden_size
        self.lstm_directions = (
            2 if self.bidirectional else 1
        )  ## decide directions based on input flag
        
        
        
        ## model layers
        # TODO initialize the look-up table.
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        # TODO assign the look-up table to the pre-trained fasttext word embeddings.
        
        self.lookup= embedding_weights   #self.embeddings
        
        ## TODO initialize lstm layer
        self.bi_lstm = torch.nn.LSTM(self.embedding_size, self.lstm_hidden_size, 
                            lstm_layers, batch_first=True , bias= True,bidirectional=True)
        
        ## TODO initialize self attention layers
        self.SelfAtt= SelfAttention(2*self.lstm_hidden_size, self_attention_config['hidden_size'],
                                    self_attention_config['output_size'])
        
        #Initialize fully connected layer
        self.fc = nn.Linear(2*self.lstm_hidden_size * self_attention_config['output_size'], self.fc_hidden_size )
        self.tanh = nn.Tanh()
        ## incase we are using bi-directional lstm we'd have to take care of bi-directional outputs in
        ## subsequent layers
        
        

    def init_hidden(self, batch_size):
        """
        Initializes hidden and context weight matrix before each
                forward pass through LSTM
        """
        h0 = torch.randn(self.lstm_directions*self.lstm_layers, batch_size, self.lstm_hidden_size)
        c0 = torch.randn(self.lstm_directions*self.lstm_layers, batch_size, self.lstm_hidden_size)
        
        return h0, c0 

    def forward_once(self, batch, lengths):
        """
        Performs the forward pass for each batch
        """

        ## batch shape: (batch_size, seq_len)
        batch_size , sequence_len = batch.size()
        ## embeddings shape: ( batch_size, seq_len, embedding_size)
        
        #h_init,c_init = self.init_hidden(batch_size)
        input_batch_sequences= self.lookup[batch]
        
        output, (hn, cn) = self.bi_lstm(input_batch_sequences, (self.h_init, self.c_init))

        return output , (hn , cn)

    def forward(self, sent1_batch, sent2_batch, sent1_lengths, sent2_lengths):
        """
        Performs the forward pass for each batch
        """
        
        
        ## init context and hidden weights for lstm cell
        self.h_init,self.c_init = self.init_hidden(self.batch_size)
        output1,_ = self.forward_once(sent1_batch,sent1_lengths)
        self.h_init,self.c_init = self.init_hidden(self.batch_size)
        output2,_ = self.forward_once(sent2_batch,sent2_lengths)
        
        ## Self attention Layer
        attended_embeddings_sent1, attention_matrix_sent1 = self.SelfAtt.forward(output1)
        attended_embeddings_sent2, attention_matrix_sent2 = self.SelfAtt.forward(output2)
        
        ## Fully connected layer 
        
        final_embeddings_sent1= self.tanh(self.fc(attended_embeddings_sent1.reshape(output1.size(0),-1)))
        final_embeddings_sent2= self.tanh(self.fc(attended_embeddings_sent2.reshape(output2.size(0),-1)))
        
        
        #similarity score prediction
        predictions = similarity_score(final_embeddings_sent1, final_embeddings_sent2)
        
        print(torch.cat((attention_matrix_sent1, attention_matrix_sent2), 1).size())
        return predictions , 

class SelfAttention(nn.Module):
    """
    Implementation of the attention block
    """

    def __init__(self, input_size, hidden_size, output_size):
        super(SelfAttention, self).__init__()
        # TODO implement
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.ws1 = nn.Linear(input_size, hidden_size, bias=False)
        self.ws2 = nn.Linear(hidden_size, output_size, bias=False)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
        
        
    ## the forward function would receive lstm's all hidden states as input
    def forward(self, attention_input):
        # TODO implement
        #pass
        
        size = attention_input.size()
        inp = attention_input.reshape(size[0]*size[1],size[2])
        attention_matrix = self.softmax(self.ws2(self.tanh(self.ws1(inp))))
        attention_matrix= attention_matrix.reshape(size[0], self.output_size, -1)
        attended_embeddings_sent1= torch.bmm(attention_matrix , attention_input)
        
        return attended_embeddings_sent1, attention_matrix

In [ ]:
batch_size = 64
output_size = 1
hidden_size = 128
vocab_size = len(vocab)
embedding_size = 300
embedding_weights = vocab.vectors
lstm_layers = 4
learning_rate = 1e-1
fc_hidden_size = 64
max_epochs = 5
bidirectional = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## self attention config
self_attention_config = {
    "hidden_size": 150,  ## refers to variable 'da' in the ICLR paper
    "output_size": 20,  ## refers to variable 'r' in the ICLR paper
    "penalty": 0.0,  ## refers to penalty coefficient term in the ICLR paper
}

In [ ]:
## init siamese lstm
siamese_lstm_attention = SiameseBiLSTMAttention(
    batch_size=batch_size,
    output_size=output_size,
    hidden_size=hidden_size,
    vocab_size=vocab_size,
    embedding_size=embedding_size,
    embedding_weights=embedding_weights,
    lstm_layers=lstm_layers,
    self_attention_config=self_attention_config,
    fc_hidden_size=fc_hidden_size,
    device=device,
    bidirectional=bidirectional,
)
## move model to device
optimizer = torch.optim.Adam(params=siamese_lstm_attention.parameters())
siamese_lstm_attention.to(device)

In [ ]:
siamese_lstm_attention.forward(batch[0],batch[1],7,4)

In [ ]:
import torch
from torch import nn
import logging
from tqdm import tqdm
from torch.autograd import Variable


logging.basicConfig(level=logging.INFO)

"""
Script for training the neural network and saving the better models 
while monitoring a metric like accuracy etc
"""


def train_model(model, optimizer, dataloader, data, max_epochs, config_dict):
    device = config_dict["device"]
    criterion = nn.MSELoss()
    max_accuracy = 5e-1
    train_loader , val_loader , test_loader = dataloader
    dictionary_info={}
    for epoch in tqdm(range(max_epochs)):
        
        try:
            # Samples the batch
            sent1_batch, sent2_batch, sent1_lengths, sent2_lengths,targets,raw_sent1,raw_sent2= next(iter(dataloader))

        except StopIteration:
            # restart the generator if the previous generator is exhausted.
            train_generator = iter(train_loader)
            sent1_batch, sent2_batch, sent1_lengths, sent2_lengths,targets,raw_sent1,raw_sent2= next(iter(dataloader))
        
        
        predictions , attention_matrix = model.forward(sent1_batch, sent2_batch, sent1_lengths, sent2_lengths)
        optimizer.zero_grad()
        
        try:
             train_loss = criterion(predictions,targets) + attention_penalty_loss(attention_matrix, 
                                                                  config_dict['self_attention_config']['penalty'], device)
                       
        except RuntimeError:
            
            raise Exception("Model Loss gets nan values on regularization.Either remove regularization or add very small values")
        
        optimizer.step()
        
        # TODO: computing accuracy using sklearn's function
        ## acc = 
        #accuracy = (torch.argmax(predictions, axis=-1) == targets).float().mean()
        
        acc=accuracy_score(targets, predictions)
        
        ## compute model metrics on dev set
        val_acc, val_loss = evaluate_dev_set(
            model, data, criterion, dataloader, config_dict, device
        )

        
        
        if val_acc > max_accuracy:
            max_accuracy = val_acc
            logging.info(
                "new model saved")  
            
            ## save the model if it is better than the prior best
            torch.save(model.state_dict(), "{}.pth".format(config_dict["model_name"]))

        logging.info(
            "Train loss: {} - acc: {} -- Validation loss: {} - acc: {}".format(
                torch.mean(train_loss.data.float()), acc, val_loss, val_acc
            )
        )
        
        if epoch % 100 == 0:
              print('[%d/%d] loss: %.3f, accuracy: %.3f' %
                   (i , max_epochs - 1, loss.item(), acc.item()))
        if epoch == max_epochs - 1:
               print('Final accuracy: %.3f, expected %.3f' %
                         (accuracy.item(), 1.0))
        
    return model


def evaluate_dev_set(model, data, criterion, data_loader, config_dict, device):
    """
    Evaluates the model performance on dev data
    """
    logging.info("Evaluating accuracy on dev set")

    # TODO implement
    pass

def attention_penalty_loss(annotation_weight_matrix, penalty_coef, device):
    """
    This function computes the loss from annotation/attention matrix
    to reduce redundancy in annotation matrix and for attention
    to focus on different parts of the sequence corresponding to the
    penalty term 'P' in the ICLR paper
    ----------------------------------
    'annotation_weight_matrix' refers to matrix 'A' in the ICLR paper
    annotation_weight_matrix shape: (batch_size, attention_out, seq_len)
    """
    
    batch_size, attention_out_size = annotation_weight_matrix.size(0), annotation_weight_matrix.size(1)
    annotation_weight_matrix_trans = torch.transpose(annotation_weight_matrix, 0, 1)
    identity = torch.eye(annotation_weight_matrix.size(0))
    annotation_mul_difference=annotation_weight_matrix@annotation_weight_matrix_trans - identity
    penalty = frobenius_norm(annotation_mul_difference)
    return penalty_coef*penalty


def frobenius_norm(annotation_mul_difference):
    """
    Computes the frobenius norm of the annotation_mul_difference input as matrix
    """
   
    """
    
 
    Args:
      annotation_mul_difference= ||AAT - I||
 
    Returns:
            regularized value
 
       
        """
    
#    torch.norm(annotation_mul_difference.float(), p='fro')
#     torch.sum(torch.sum(torch.sum(annotation_mul_difference**2,1),1)**0.5).type(torch.DoubleTensor)
    return torch.sqrt(torch.sum(annotation_mul_difference**2))

In [ ]:
siamese_lstm_attention = train_model(
    model=siamese_lstm_attention,
    optimizer=optimizer,
    dataloader=train_loader,
    data=sick_data,
    max_epochs=max_epochs,
    config_dict={
        "device": device,
        "model_name": "siamese_lstm_attention",
        "self_attention_config": self_attention_config,
    },
)

In [ ]:
train_dataset=  STSDataset(train_data_df['sent1_tensor'],
                         train_data_df['sent2_tensor'],
                         train_data_df['relatedness_score'],
                         train_data_df['sents1_length_tensor'],
                         train_data_df['sents2_length_tensor'],
                         train_data_df['sentence_A'],
                         train_data_df['sentence_B']
                         )

train_loader = DataLoader(dataset = train_dataset, batch_size = 64, shuffle = True)

In [ ]:
# main training loop
# train_loader , val_loader , test_loader = sick_dataloaders
train_generator = iter(train_loader)
for i in range(100):
    try:
        # Samples the batch
        sent1_batch, sent2_batch, sent1_lengths, sent2_lengths, targets,raw_sent1,raw_sent2= next(train_generator)
        print(Variable(targets))
        break
    except StopIteration:
        # restart the generator if the previous generator is exhausted.
        train_generator = iter(train_loader)
        sent1_batch, sent2_batch, sent1_lengths, sent2_lengths, targets,raw_sent1,raw_sent2= next(train_generator)

In [ ]:
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])

In [ ]:
vocab.vectors[input]
embedding = nn.Embedding(len(vocab), 300)
embedding(input)

In [ ]:
vocab.vectors.size()

In [ ]:
# with torch.no_grad():
a=torch.randn([64, 40, 14])
at= np.transpose(a, (0, 2, 1)).clone().detach().requires_grad_(True)
# frobenius_norm(a)
diff=a@at-torch.eye(a.size(1))
diff.size()

In [ ]:
att=torch.randn([64, 40, 14])
attT = att.transpose(1,2)
identity = torch.eye(att.size(1))
identity = Variable(identity.unsqueeze(0).expand(train_loader.batch_size,att.size(1),att.size(1)))
diff=att@attT - identity
diff.size()

In [ ]:
identity

In [ ]:
t1=torch.tensor([0.3724, 0.4367, 0.3584, 0.4131, 0.4151, 0.3754, 0.4444, 0.3806, 0.4540,
        0.3293, 0.4363, 0.3715, 0.4190, 0.4442, 0.4099, 0.4260, 0.3768, 0.4290,
        0.4401, 0.4283, 0.3629, 0.3899, 0.3863, 0.4223, 0.4615, 0.4136, 0.3793,
        0.4060, 0.4051, 0.4540, 0.4548, 0.4677, 0.4346, 0.3802, 0.3646, 0.4234,
        0.4135, 0.3769, 0.4404, 0.3910, 0.4340, 0.4046, 0.3871, 0.3655, 0.4430,
        0.4434, 0.3824, 0.4189, 0.4324, 0.3946, 0.3496, 0.4714, 0.4147, 0.4114,
        0.4263, 0.4088, 0.3955, 0.3722, 0.4222, 0.3962, 0.3961, 0.4276, 0.3328,
        0.3668]) 
t2=torch.tensor([0.7000, 0.1750, 0.5750, 0.8250, 0.3250, 0.4750, 0.8500, 0.7750, 0.5750,
        0.6000, 0.5750, 0.4500, 0.6500, 0.6750, 0.7750, 0.9250, 0.1000, 0.8750,
        0.5250, 0.6500, 0.5750, 0.3000, 0.0000, 0.6750, 0.0250, 0.7750, 0.9500,
        0.6250, 0.3250, 0.9500, 0.6250, 0.5750, 0.7250, 0.4000, 0.9000, 0.7250,
        0.9750, 0.0000, 0.5500, 0.6000, 0.8250, 0.2500, 0.7000, 0.9750, 0.8750,
        0.5750, 0.6750, 0.6500, 0.0750, 0.9000, 0.1500, 0.8750, 0.7250, 0.6500,
        0.8250, 0.5750, 0.5000, 0.7750, 0.5250, 0.0250, 0.9000, 0.5750, 0.7500,
        0.0500])
# tensor(0.1044, dtype=torch.float64, grad_fn=<AddBackward0>)

In [ ]:
# accuracy_score(list(t1.numpy()),list(t2.numpy()))

In [ ]:
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.metrics import pairwise.cosine_similarity,DistanceMetric , r2_score , max_error, mean_absolute_error, explained_variance_score
r2_score(list(t2),list(t1))
# DistanceMetric(list(t2),list(t1))

In [ ]:
explained_variance_score(t1,t2)

In [ ]:
mean_absolute_error(t1,t2)
-1.7976931348623157e+308

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(t1, t2, dense_output=True)

In [ ]:
train_loader.batch_sampler

In [ ]:
len(train_loader)

In [ ]:
70*64


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
x = np.linspace(0, 5, 10)
y = x ** 2

fig, axes = plt.subplots(1, 2, figsize=(8,4), dpi=100)

# plot subplot 1
axes[0].plot(x, x**2, color="green", label="y = x**2")
axes[0].plot(x, x**3, color="red", label="y = x**3")
axes[0].legend(loc=2); # upper left corner
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('Plot of y=x^2 and y=x^3')

# plot subplot 2
axes[1].plot(x, x**2, color="violet", label="y = x**2")
axes[1].plot(x, x**3, color="blue", label="y = x**3")
axes[1].legend(loc=2); # upper left corner
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_title('Plot of y=x^2 and y=x^3')

# `fig.tight_layout()` automatically adjusts the positions of the axes on the figure canvas so that there is no overlapping content
# comment this out to see the difference
fig.tight_layout()

plt.show()

In [ ]:
train=train_pd
train_pd['relatedness_score']=(train_pd['relatedness_score']-train_pd['relatedness_score'].min())/(train_pd['relatedness_score'].max()-train_pd['relatedness_score'].min())

In [7]:
for batch in range(batch_train_num):
            
            try:
                # Samples the batch
                sent1_batch, sent2_batch, sent1_lengths, sent2_lengths,targets,raw_sent1,raw_sent2= next(train_generator)

            except StopIteration:
                # restart the generator if the previous generator is exhausted.
                train_generator = iter(train_loader)
                sent1_batch, sent2_batch, sent1_lengths, sent2_lengths,targets,raw_sent1,raw_sent2= next(train_generator)
        

NameError: name 'batch_train_num' is not defined

In [8]:
load_dataset

<function datasets.load.load_dataset(path: str, name: Union[str, NoneType] = None, data_dir: Union[str, NoneType] = None, data_files: Union[Dict, List] = None, split: Union[str, datasets.splits.Split, NoneType] = None, cache_dir: Union[str, NoneType] = None, features: Union[datasets.features.Features, NoneType] = None, download_config: Union[datasets.utils.file_utils.DownloadConfig, NoneType] = None, download_mode: Union[datasets.utils.download_manager.GenerateMode, NoneType] = None, ignore_verifications: bool = False, keep_in_memory: Union[bool, NoneType] = None, save_infos: bool = False, script_version: Union[str, datasets.utils.version.Version, NoneType] = None, use_auth_token: Union[str, bool, NoneType] = None, task: Union[str, datasets.tasks.base.TaskTemplate, NoneType] = None, streaming: bool = False, **config_kwargs) -> Union[datasets.dataset_dict.DatasetDict, datasets.arrow_dataset.Dataset, datasets.dataset_dict.IterableDatasetDict, datasets.iterable_dataset.IterableDataset]>

In [ ]:
optimizer.zero_grad(req)